In [1]:
# Importing the required module
# TODO: Use other plotting library instead?
# TODO: Check which libraries remain relevant at the end.
import numpy as np

import pandas as pd


import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objs as go

import os

from functions_data_gen import *



# dirname = os.path.dirname()

# Define own working directory here 
# os.chdir(r"/Users/philipp/econ/experiment_model_persuasion/")


pwd = os.getcwd()
pwd

'/Users/philipp/econ/experiment_model_persuasion/src/sandbox'

In [2]:
# Test data generation 

# Set parameters
T=100

#Generate timeline

t = np.arange(0,T,1,dtype=int)

# Set std and switching probability

var_errors = 30
p = 0.5

# Generate (stochastic) trend:

seed = 12345

rng = np.random.default_rng(seed)
draw_rand_bin = rng.binomial(1, p, size=None)

errors = rng.normal(0, var_errors, T)




# Set the trend component
c = .5

# Generate trend shift (first up, then down, or vice versa)

trend = np.array([0,0])

if draw_rand_bin == 0:
    trend = np.array([c,-c])
elif draw_rand_bin == 1:
    trend = np.array([c,-c])    


# Determine whether there is a trend shift
# For shift at t=0, this is equivalent to no shift (i.e., second part of trend is used throughout).

# Set up shifting probabilities
p_shift_at_0 = 0.5

equal_prob_t80= (1-  p_shift_at_0) / (79)

p_shift_at_t = np.full(80, equal_prob_t80)


p_shift_at_t[0] = p_shift_at_0

p_shift_at_t

check_sum = np.sum(p_shift_at_t)
# check_sum


trend_shift = np.random.choice(80, size=None, replace=True, p=p_shift_at_t.tolist())

trend_shift = np.random.choice(80, size=None, replace=True)

trend_shift

# generate trend vector including shift

trend_with_shift = np.full(100,c)
for i in range(0,100):
    if i <= trend_shift:
        trend_with_shift[i] = trend[0]
    elif i > trend_shift:
        trend_with_shift[i] = trend[1]

trend_with_shift


# run cumulative trend, errors

cum_sum_trend = np.cumsum(trend_with_shift)

cum_sum_errors = np.cumsum(errors)

# shift y upwards by 100 to "normalize" stock
y = cum_sum_trend + cum_sum_errors + 100

stacked_array= np.stack((t, y), axis=-1)
data = pd.DataFrame(stacked_array).rename(columns={0: "Time", 1: "Stock price"})

data.iloc[50]

# Normalize stock price level



data["Stock price"] = data["Stock price"] - data["Stock price"].min() + 100


In [10]:
# TODO: Make graphs, regressions look nice.
# Challenge: retain uniformity in order not to bias 
# experimental persuaders/receivers.

# TODO: Does plotly require pandas dataframes, or are numpy-array fine, too?
# Documentation indicates generally pd. fine.

# TODO: Find a dynamic and consistent way to set the axes (y, mostly)


shift_point = 40

fig = px.line(data.head(80), x="Time", 
                        y="Stock price",
                        labels={'number_obs':'Number of Observations', 'runtime':'runtime'},
                        title='Development of a stock price',
                        )
fig.update_layout(paper_bgcolor='#fff' )
fig.update_layout(plot_bgcolor='#fff' )

fig.add_shape(type="line",
    xref="x", yref="y",
    x0=shift_point, y0=0, x1=shift_point, y1=data["Stock price"].max()+100,
    line=dict(
        color="LightSeaGreen",
        width=3,
    ),
)

fig.add_shape(type="line",
    xref="x", yref="y",
    x0=shift_point, y0=data.iat[shift_point,1], x1=80, y1=data.iat[80,1],
    line=dict(
        color="LightSeaGreen",
        width=3,
    ),
)

fig.add_shape(type="line",
    xref="x", yref="y",
    x0=0, y0=data.iat[0,1], x1=shift_point, y1=data.iat[shift_point,1],
    line=dict(
        color="LightSeaGreen",
        width=3,
    ),
)
fig.update_yaxes(range=[0, data["Stock price"].max()+100])
# fig.to_image(format="png", engine="kaleido")

# fig.write_image(pwd + "/img/fig_test_{}.jpg".format(shift_point), engine='kaleido')



In [4]:
#from gen_neutral_graphs import gen_neutral_graphs

import plotly.express as px

from functions_plotting import *

T_model = 79 

t_list_model = np.arange(0,T_model,1,dtype=int)


gen_model_graphs_trunc(data, t_list_model, 81, "/Users/philipp/econ/experiment_model_persuasion/" )

In [7]:
gen_neutral_graphs_trunc(data, 80, "/Users/philipp/econ/experiment_model_persuasion/")

In [3]:


data_test = generate_data(model_switching_point=40, model_change_sign="pos_to_neg", trend_abs=0.5, var_errors=30, obs=100, seed=12345)



In [6]:
data_test_2 = generate_data(model_switching_point=20, model_change_sign="pos_to_neg", trend_abs=1, var_errors=10, obs=100, seed=55555)

data_test_2

test_min = np.amin(data_test_2["Stock price"])

data_test_2["Stock price"] = data_test_2["Stock price"] - test_min + 100

data_test_2


,Time,Stock price
0,0.0,196.606273
1,1.0,212.463408
2,2.0,217.542707
3,3.0,218.505007
4,4.0,209.386863
...,...,...
95,95.0,107.087803
96,96.0,102.771057
97,97.0,112.031897
98,98.0,101.654953


In [7]:
from functions_plotting import * 
gen_neutral_graphs_trunc(data=data_test_2, trunc=80, wd="/Users/philipp/econ/experiment_model_persuasion/")

### ToDos:

- Generate data five times, make five okay looking graphs.

- cut them to 80, generate neutral graphs
    - "trunc" functions do this now

- present 79? model messages

- save  them